> 这次学习了如何定义模型的三种方式Sequential，ModuleList和ModuleDict，
以U-Net为例，了解如何搭建模型，
修改resnet模型的若干层，添加额外输入和输出，
如何存储和加载模型，以及在多卡和单卡环境保存与加载的方法

### three methods about model define 

#### 1.1 Sequential

对应模块为nn.Sequential()。

当模型的前向计算为简单串联各个层的计算时， 
Sequential 类可以通过更加简单的方式定义模型。
它可以接收一个子模块的有序字典(OrderedDict) 或者
一系列子模块作为参数来逐一添加 Module 的实例，
⽽模型的前向计算就是将这些实例按添加的顺序逐⼀计算。
我们结合Sequential和定义方式加以理解：


In [ ]:
class MySequential(nn.Module):
    from collections import OrderedDict
    def __init__(self, *args):
        super(MySequential, self).__init__()
        if len(args) == 1 and isinstance(args[0], OrderedDict): # 如果传入的是一个OrderedDict
            for key, module in args[0].items():
                self.add_module(key, module)  # add_module方法会将module添加进self._modules(一个OrderedDict)
        else:  # 传入的是一些Module
            for idx, module in enumerate(args):
                self.add_module(str(idx), module)
    def forward(self, input):
        # self._modules返回一个 OrderedDict，保证会按照成员添加时的顺序遍历成
        for module in self._modules.values():
            input = module(input)
        return input

下面来看下如何使用Sequential来定义模型。

只需要将模型的层按序排列起来即可，根据层名的不同，排列的时候有两种方式：
- 直接排列

In [2]:
import torch.nn as nn
net = nn.Sequential(
    nn.Linear(336,256),
    nn.ReLU(),
    nn.Linear(256,2)
)
print(net)

Sequential(
  (0): Linear(in_features=336, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=2, bias=True)
)


* 使用OrderedDict：

In [4]:
import collections
import torch.nn as nn
net2=nn.Sequential(collections.OrderedDict([
    ('fc1',nn.Linear(336,256)),
    ('relu1',nn.ReLU()),
    ('fc2',nn.Linear(256,2))
]))
print(net2)

Sequential(
  (fc1): Linear(in_features=336, out_features=256, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=256, out_features=2, bias=True)
)


Sequential定义模型的好处在于简单、易读,不需要再写forward，因为顺序已经定义好了

但使用Sequential也会使得模型定义丧失灵活性.比如需要在模型中间加入一个外部输入时就不适合用Sequential的方式实现。使用时需根据实际需求加以选择。

#### 1.3 ModuleList

对应模块为nn.ModuleList()

ModuleList 接收一个子模块（或层，需属于nn.Module类）的列表作为输入，然后也可以类似List那样进行append和extend操作。同时，子模块或层的权重也会自动添加到网络中来。

In [3]:
net = nn.ModuleList([nn.Linear(336,256),nn.ReLU()])
net.append(nn.Linear(256,2))
print(net[-1])
print(net)

Linear(in_features=256, out_features=2, bias=True)
ModuleList(
  (0): Linear(in_features=336, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=2, bias=True)
)


要特别注意的是，nn.ModuleList 并没有定义一个网络，
它只是将不同的模块储存在一起。

ModuleList中元素的先后顺序并不代表其在网络中的真实位置顺序，

需要经过forward函数指定各个层的先后顺序后才算完成了模型的定义。

具体实现时用for循环即可完成：

#### 1.4 ModuleDict

对应模块为nn.ModuleDict()

ModuleDict和ModuleList的作用类似，只是ModuleDict能够方便添加层名称

In [7]:
net = nn.ModuleDict({
    'linear': nn.Linear(336,256),
    'act':nn.ReLU()
})

net['output']=nn.Linear(256,2)
print(net['linear'])
print(net.output)
print(net)

Linear(in_features=336, out_features=256, bias=True)
Linear(in_features=256, out_features=2, bias=True)
ModuleDict(
  (linear): Linear(in_features=336, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=2, bias=True)
)


### 2. 利用模型块快速搭建复杂网络

在实际场景下不一定利于使用。当模型的深度非常大时候，
使用Sequential定义模型结构需要向其中添加几百行代码，使用起来不甚方便。

对于大部分模型结构（比如ResNet、DenseNet等），我们仔细观察就会发现，虽然模型有很多层， 但是其中有很多重复出现的结构。考虑到每一层有其输入和输出，若干层串联成的”模块“也有其输入和输出，如果我们能将这些重复出现的层定义为一个”模块“，每次只需要向网络中添加对应的模块来构建模型，这样将会极大便利模型构建的过程。

本节我们将以U-Net为例，介绍如何构建模型块，以及如何利用模型块快速搭建复杂模型。

经过本节的学习，你将收获：

* 利用上一节学到的知识，将简单层构建成具有特定功能的模型块
* 利用模型块构建复杂网络

#### 2.1 U-Net简介

U-Net是分割 (Segmentation) 模型的杰作，在以医学影像为代表的诸多领域有着广泛的应用。U-Net模型结构如下图所示，通过残差连接结构解决了模型学习中的退化问题，使得神经网络的深度能够不断扩展。

![unet](./figures/5.2.1unet.png)

#### U-Net模型块分析
结合上图，不难发现U-Net模型具有非常好的对称性。模型从上到下分为若干层，每层由左侧和右侧两个模型快组成，每侧的模型快与其上下模型块之间有连接；
同时位于同一层左右两侧的模型快之间也有连接，称为“Skip-connection”。此外还有输入和输出处理等其它组成部分。由于模型的形状非常像英文字母的“U”，因此被命名为“U-Net”


组成U-Net的模型块主要有如下几个部分：
1） 每个子块内部的两次卷积（Double Convolution）
2） 左侧模型块之间的下采样连接，即最大池化（Max pooling）
3） 右侧模型块之间的上采样连接（Up sampling）
4） 输出层的处理
除模型块外，还有模型块之间的横向连接，输入和U-Net底部的连接等计算，这些单独的操作可同通过forward函数实现


#### U-Net模型块实现

不必把每一层按序排列显示出来，且不易读。

比较好的方法是先定义好模型，再定义模型块之间的连接顺序和计算方式。就好比装配零件一样，先装配好一些基础的部件，再用这些可以复用的部件得到整个装配体


这里的基础部件对应上一节分析的四个模型块，根据功能我们将其命名为：DoubleConv,Down, Up,OutConv

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [9]:
class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self,in_channels,out_channels,mid_channels=None):
        super().__init()
        if not mid_channels:
            mid_channels=out_channels
        self.double_conv=nn.Sequential(
            nn.Conv2d(in_channels,mid_channels,kernel_size=3,padding=1,bias=False),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels,out_channels,kernel_size=3,padding=1,bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self,x):
        return self.double_conv(x)

In [ ]:
class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self,in_channels,out_channels):
        super().__init__
        self.maxpool_conv=nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels,out_channels)
        )
    
    def forward(self,x):
        return self.maxpool_conv(x)

In [10]:
class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self,in_channels,out_channels,bilinear=True):
        super().__init__

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up=nn.Upsample(scale_factor=2,mode="bilinear",align_corners=True)
            self.conv=DoubleConv(in_channels,out_channels,in_channels//2)
        else:
            self.up=nn.ConvTranspose2d(in_channels,in_channels//2,kernel_size=2,stride=2)
            self.conv=DoubleConv(in_channels,out_channels)
        
    def forward(self,x1,x2):
        x1=self.up(x1)
        # input is CHW
        diffY=x2.size()[2]-x1.size()[2]
        diffX=x2.size()[3]-x1.size()[3]

        x1=F.pad(x1,[diffX//2,diffX-diffX//2,
                    diffY//2,diffY-diffY//2])
        
        # if you havepadding issues,see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x=torch.cat([x2,x1],dim=1)
        return self.conv(x)


In [11]:
class OutConv(nn.Module):
    def __init__(self,in_channels,out_channels):
        super(OutConv,self).__init__()
        self.conv=nn.Conv2d(in_channels,out_channels,kernel_size=1)
    
    def forward(self,x):
        return self.conv(x)
        

#### 2.4 利用模型块组装U-Net


In [12]:
class UNet(nn.Module):
    def __init__(self,n_channels,n_classes,bilinear=True):
        super(UNet,self).__init__()
        self.n_channels=n_channels
        self.n_classes=n_classes
        self.bilinear=bilinear

        self.inc=DoubleConv(n_channels,64)
        self.down1=Down(64,128)
        self.down2=Down(128,256)
        self.down3=Down(256,512)
        factor=2 if bilinear else 1
        self.down4=Down(512,1024//factor)
        self.up1=Up(1024,512//factor,bilinear)
        self.up2=Up(512,256//factor,bilinear)
        self.up3=Up(256,128//factor,bilinear)
        self.up4=Up(128,64,bilinear)
        self.outc=OutConv(64,n_classes)
    
    def forward(self,x):
        x1=self.inc(x)
        x2=self.down1(x1)
        x3=self.down1(x2)
        x4=self.down1(x3)
        x5=self.down1(x4)
        x=self.up1(x5,x4)
        x=self.up2(x,x3)
        x=self.up3(x,x2)
        x=self.up4(x,x1)
        logits=self.outc(x)
        return logits

本节参考

https://github.com/milesial/Pytorch-UNet

### PyTorch修改模型

除了自己构建PyTorch模型外，还有另一种应用场景：我们已经有一个现成的模型，但该模型中的部分结构不符合我们的要求，为了使用模型，我们需要对模型结构进行必要的修改。随着深度学习的发展和PyTorch越来越广泛的使用，有越来越多的开源模型可以供我们使用，很多时候我们也不必从头开始构建模型。因此，掌握如何修改PyTorch模型就显得尤为重要。

如何在已有模型的基础上：
* 修改模型若干层
* 添加额外输入
* 添加额外输出

#### 修改模型层
以torchvision预定义好的模型ResNet50为例，探索如何修改模型的某一层或者某几层


In [1]:
import torchvision.models as models
net=models.resnet50()
print(net)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

模型结构是为了适配ImageNet预训练的权重，因此最后全连接层（fc）的输出节点数是1000
假设要用这个resnet模型做一个10分类的问题，最简单的就是修改模型的fc层，将输出节点替换为10，其它层不做改动。但是如果觉得全连接层可能太少了，想再加一层，可以做如下修改：

In [5]:
import torch.nn as nn
from collections import OrderedDict

classifier = nn.Sequential(OrderedDict([('fc1',nn.Linear(2048,128)),
                                        ('relu1',nn.ReLU()),
                                        ('dropout1',nn.Dropout(0.5)),
                                        ('fc2',nn.Linear(128,10)),
                                        ('output',nn.Softmax(dim=1))
                                        ]))
print(classifier)
net.fc=classifier
print(net)

Sequential(
  (fc1): Linear(in_features=2048, out_features=128, bias=True)
  (relu1): ReLU()
  (dropout1): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
  (output): Softmax(dim=1)
)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias

操作相当于将模型（net）最后名称为“fc”的层替换成了名称为“classifier”的结构，该结构是我们自己定义的。这里使用了第一节介绍的Sequential+OrderedDict的模型定义方式。
至此，我们就完成了模型的修改，现在的模型就可以去做10分类任务了。

#### 2.4 添加外部输入

有时候在模型训练中，除了已有模型的输入之外，还需要输入额外的信息。
如CNN网络中，除了输入图像，还需要同时输入图像对应的其它信息。
就需要在已有的CNN网络中添加额外的输入变量，基本思想是：将原模型添加输入位置前的部分作为一个整体，同时在forward中定义好原模型不变的部分、
添加的输入和后续层之间的连接关系，从而完成模型的修改。

还以resnet50为基础，任务还是10分类。不同点在于，我们希望利用已有的模型结构，
在倒数第二层增加一个额外的输入变量add_variable来辅助预测。具体实现如下：``

In [6]:
class Model(nn.Module):
    def __init__(self,net):
        super(Model,self).__init__()
        self.net=net
        self.relu=nn.ReLU()
        self.dropout=nn.Dropout(0.5)
        self.fc_add=nn.Linear(1001,10,bias=True)
        self.output=nn.Softmax(dim=1)
    
    def forward(self,x,add_variable):
        x=self.net(x)
        x=torch.cat((self.dropout(self.relu(x)), add_variable.unsequeeze(1)),1)
        x=self.fc_add(x)
        x=self.output(x)
        return x


实现要点是通过torch.cat实现了tensor的拼接。torchvision中的resnet50输出是一个1000维的tensor，
通过修改forward函数（配套定义一些层），先将2048维的tensor通过激活函数层和dropout层，再和外部输入变量‘add_variable’拼接，
最后通过全连接层映射到指定的输出维度10

另外这里对外部输入变量“add_variable”进行unsequeeze操作是为了和net输出的tensor保持维度一致。
常用于add_variable是单一数值（scalar）的情况。此时add_variable的维度是（batch_size,),需在第二维补充维数1，
从而可以和tensor进行torch.cat操作。

对修改好的模型结构进行实例化，就可以使用了

In [8]:
import torchvision.models as models
net=models.resnet50()
model=Model(net).cuda()


另外不能忘了，训练中在输入数据的时候要给两个inputs

In [ ]:
outputs=model(inputs,add_var)

#### 添加额外输出
有时候在模型训练追踪，除了模型最后的输出外，需要输出模型某一中间层的结果，以施加额外的监督，获得更好的中间层结果。
基本思路是修改模型定义中forward函数的return变量

依然以resnet50做10分类任务为例，在已经定义好的模型结构上，同时输出1000维的倒数第二层和10维的最后一层模型结果。
集体实现如下：

In [ ]:
class Model(nn.Module):
    def __init__(self,net):
        super(Model,self).__inti__()
        self.net=net
        self.relu=nn.ReLU()
        self.dropout=nn.Dropout(0.5)
        self.fc1=nn.Linear(1000,10,bias=True)
        self.output=nn.Softmax(dim=1)

    def forward(self,x,add_variable):
        x1000=self.net(x)
        x10=self.dropout(self.relu(x1000))
        x10=self.fc1(x10)
        x10=self.output(x10)
        return x10, x1000

之后对修改好的模型结构进行实例化，就可以使用了

In [9]:
import torchvision.models as models
net=models.resnet50()
model=Model(net).cuda()

这里需要注意输入数据后会有两个outputs

In [ ]:
out10,out1000=model(inputs,add_var)

### Pytorch模型保存与读取
在很多场景下我们都会使用多GPU训练。这种情况下，模型会分布于各个GPU上（参考2.3节分布数据式训练，这里暂不考虑分布模型式训练），
模型的保存和读取与单GPU训练情景下是否有所不同？

经过本节的学习，你将收获：

* PyTorch的模型的存储格式
* PyTorch如何存储模型
* 单卡与多卡训练下模型的保存与加载方法

#### 4.1 模型存储格式
主要采用pkl，pt，pth三种格式。就是应用层面来说没区别，这里也不做具体的讨论。

#### 4.2 模型存储内容
一个PyTorch模型主要包含两个部分：模型结构和权重。其中模型是继承nn.Module的类，权重的数据结构是一个字典（key是层名，value是权重向量）。存储也分为两种形式，存储整个模型（包括结构和权重），和只存储模型权重

In [ ]:
from torchvision import models
model=models.resnet152(pretrained=True)

# 保存整个模型
torch.save(model,save_dir)
# 保存模型权重
torch.save(model.state_dict,save_dir)

对PyTorch而言，pt,pth和pkl三种数据格式均支持模型权重和整个模型的存储，因此使用上没有差别

#### 4.3 单卡和多卡模型存储的区别

Pytorch中将模型和数据放到GPU上有两种方式——— .cuda()和.to(device)。本节后续内容针对前一种方式进行讨论。如果要使用多卡训练的话，需要对模型使用torch.nn.DataParallel。示例如下：

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES']='0' # 如果是多卡改成类似0,1,2
model=model.cuda() #单卡
model=torch.nn.DataParallel(model).cuda() #多卡

之后我们把model对应的layer名称打印出来，可以观察到差别在于多卡并行的模型每层的名称前多了一个“module”

- 单卡模型的层名


![img](https://pic3.zhimg.com/v2-3490f6ab8bc806274dd017e1a66e2486_b.png)

- 多卡模型的层名：

![img](https://pic3.zhimg.com/v2-4b611c24c2e702749cebbe65eaff7cde_b.png)

这种模型表示的不同可能会导致模型保存和加载过程中需要处理一些矛盾点，下面对各种可能的情况做分类讨论

#### 4.4情况分类讨论
由于训练和测试所使用的硬件条件不同，在模型的保存和加载过程中可能因为单GPU和多GPU环境的不同带来模型不匹配等问题。
这里对PyTorch框架下单卡/多卡下模型的保存和加载问题进行排列组合（=4），样例模型是torchvision中预训练模型resnet152

* 单卡保存+单卡加载

在使用os.envision命令指定使用的GPU后，即可以进行模型保存和读取操作。注意这里即便保存和读取使用的GPU不同也无妨


In [ ]:
import os
import torch
from torchvision import models

os.environ['CUDA_VISIBLE_DEVICES']='0' # 这里替换成希望使用的GPU编号
model=models.resnet152(pretrained=True)
model.cuda()

# 保存+读取整个模型
torch.save(model,save_dir)
loaded_model=torch.load(save_dir)
loaded_model.cuad()

# 保存+读取模型权重
torch.save(model.state_dict(),save_dir)
loaded_dict=torch.load(save_dir)
loaded_model=models.resnet152() # 注意这里需要对模型结构有定义
loaded_model.state_dict=loaded_dict
loaded_model.cuda()

* 单卡保存+多卡加载

这种情况处理比较简单，读取单卡保存的模型后，使用nn.DataParallel函数进行分布式训练设置即可（相当于3章节1代码中.cuda()替换一下）：

In [ ]:
import os
import torch
from torchvision import models

os.environ['CUDA_VISIBLE_DEVICES']='0' 
model=models.resnet152(pretrained=True)
model.cuda()

# 保存+读取整个模型
torch.save(model,save_dir)

os.environ['CUDA_VISIBLE_DEVICES']='1,2'
loaded_model=torch.load(save_dir)
loaded_model=nn.DataParallel(loaded_model).cuda()

# 保存+读取模型权重
torch.save(model.state_dict(),save_dir)

os.environ['CUDA_VISIBLE_DEVICES']='1,2'
loaded_dict=torch.load(save_dir)
loaded_model=models.resnet152()
loaded_model.state_dict=loaded_dict
loaded_model=nn.DataParallel(loaded_model).cuda()

* 多卡保存+单卡加载

这种情况下的核心问题是：如何去掉权重字典键名中的“module”，以保证模型的统一性。

对于加载整个模型，直接提取模型的module属性即可：

In [ ]:
import os
import torch
from torchvision import models
os.environ['CUDA_VISIBLE_DEVICES']='1,2'

model=models.resnet152(pretrained=True)
model=nn.DataParallel(model).cuda()

# 保存+读取整个模型
torch.save(model,save_dir)

os.environ['CUDA_VISIBLE_DEVICES']='0'
loaded_model=torch.load(save_dir)
loaded_model=loaded_model.module

对于加载模型权重，有以下几种思路：

**去除字典里的module麻烦，往model里添加module简单（推荐）**

In [ ]:
import os
import torch
from torchvision import models

os.environ['CUDA_VISIBLE_DEVICES']='0,1,2'

model=models.resnet152(pretrained=True)
model=nn.DataParallel(model).cuda()

# 保存+读取模型权重
torch.save(model.state_dict(),save_dir)

os.environ['CUDA_VISIBLE_DEVICES']='0'
loaded_dict=torch.load(save_dir)
loaded_model=models.resnet152()
loaded_model=nn.DataParallel(loaded_model).cuda()
loaded_model.state_dict=loaded_dict

这样即使是单卡，也可开始训练（相当于分布到单卡上）

**遍历字典去除module**

In [ ]:
from collections import OrderedDict
os.environ['CUDA_VISIBLE_DEVICES']='0'

loaded_dict=torch.load(save_dir)
for k,v in loaded_dict.item():
    name=k[7:] # model字段在最前面，从第7个字符开始就可以去掉module
    new_state_dict[name]=v # 新字典的key值对应的value一一对应
loaded_model=models.resnet152()
loaded_model.state_dict=new_state_dict
loaded_model=loaded_model.cuda()

**使用replace操作去除module**

In [ ]:
loaded_model=models.resnet152()
loaded_dict=torch.load(save_dir)
loaded_model.load_state_dict({k.replace('module.',''): v for k,v in loaded_dict.items()})

* 多卡保存+多卡加载

由于是模型保存和加载都使用的是多卡，因此不存在模型层名前缀不同的问题。
单多卡状态下存在一个device（使用的GPU）匹配的问题，即保存整个模型是会同时保存所使用的GPU id等信息，
读取时若这些信息和当前使用的GPU信息不符则可能会报错或者程序不按预定状态运行。具体表现为以下两点：

**读取整个模型再使用nn.DataParallel进行分布式训练设置**

这种情况很可能会造成保存的整个模型GPU id和读取环境下设置的GPU id不符，训练时数据所在device和模型所在device不一致而报错

**读取整个模型而不使用nn.DataParallel进行分布式训练设置**

这种情况可能不会报错，测试中发现程序会自动使用设备的前n个GPU进行训练（n是保存的模型使用的GPU个数）。此时如果指定的GPU个数少于n，则会报错。在这种情况下，只有保存模型时环境的device id和读取模型时环境device id一致，程序才会按照预期在指定的GPU上进行分布式训练。

相比之下，读取模型权重，之后再使用nn.DataParallel进行分布式训练设置则没有问题。因此多卡模型下建议使用的权重的方式存储和读取模型：



In [ ]:
import os
import torch
from torchvision import models

os.environ['CUDA_VISIBLE_DEVICES']='0,1,2'

model=models.resnet152(pretrained=True)
model=nn.DataParallel(model).cuda()

# 保存+读取模型权重，强烈建议！
torch.save(model.state_dict(),save_dir)
loaded_dict=torch.load(save_dir)
loaded_model=models.resent152()
loaded_model=nn.DataParallel(loaded_model).cuda()
loaded_model.state_dict=loaded_dict

如果只有保存的整个模型，也可以采用提取权重的方式构建新的模型

In [ ]:
# 读取整个模型
loaded_whole_model=torch.load(save_dir)
loaded_model=models.resent152()
loaded_model.state_dict=loaded_whole_model.state_dict
loaded_mo=nn.DataParallel(loaded_model).cuda()